In [3]:
pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [6]:
df=pd.read_json("https://raw.githubusercontent.com/Zaniiiii/Data/main/pukul6/bias/berlabel/dataset_bias_detik_berlabel.json",lines=True)

In [ ]:
df

In [ ]:
df['content'] = df['content'].str.split('Baca selengkapnya di').str[0]

df

In [ ]:
#Hapus  ADVERTISEMENT SCROLL TO CONTINUE WITH CONTENT, :[Gambas:Video 20detik]
def clean_content(content):
    # Remove \r, \n, and any other unwanted characters
    content = re.sub(r'[\r\n]', ' ', content)

    # Remove multiple spaces
    content = re.sub(r'\s+', ' ', content).strip()
    content = content.replace("[Gambas:Video 20detik]", "")
    content = content.replace("ADVERTISEMENT SCROLL TO CONTINUE WITH CONTENT", "")
    content = content.replace("ADVERTISEMENT", "")
    return content.strip()

# Apply the cleaning function to the dataframe
df["content"] = df["content"].apply(clean_content)
df

In [11]:
import nltk
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import re
from tqdm import tqdm
from multiprocessing import Pool
from functools import lru_cache

nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [1]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'\W', ' ', text)
    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Stemming
    # text = stemmer.stem(text)
    return text


In [13]:
import pandas as pd
import time
from multiprocessing import Pool

# Fungsi untuk membersihkan teks dengan progress bar
def clean_text_with_progress(x,y):
    start_time = time.time()

    # Menggunakan multiprocessing Pool untuk membersihkan teks secara paralel
    with Pool() as pool:
        cleaned_content = list(tqdm(pool.imap(clean_text, x), total=len(df), desc="Cleaning texts", unit="text"))

    end_time = time.time()
    total_time = end_time - start_time
    avg_time_per_row = total_time / len(df)

    df[y] = cleaned_content
    print("Estimasi waktu pembersihan per baris: {:.6f} detik".format(avg_time_per_row))
    print("Total waktu pembersihan: {:.6f} detik".format(total_time))

clean_text_with_progress(df["content"],"content")
clean_text_with_progress(df["title"],"title")

Cleaning texts: 100%|██████████| 100/100 [05:29<00:00,  3.30s/text]

Estimasi waktu pembersihan per baris: 3.300061 detik
Total waktu pembersihan: 330.006056 detik



Cleaning texts: 100%|██████████| 100/100 [00:21<00:00,  4.60text/s]

Estimasi waktu pembersihan per baris: 0.218276 detik
Total waktu pembersihan: 21.827584 detik


In [20]:
df["platform"] = "detik"
df

,title,link,date,content,is_fake,media_bias,bias,platform
0,polisi tunggu tuju keluarga korban pesawat jat...,https://news.detik.com/berita/d-7348286/polisi...,2024-05-19,jenazah korban pesawat latih jatuh bsd serpong...,0,netral,0,detik
1,rs polri kondisi korban tewas pesawat jatuh bs...,https://news.detik.com/berita/d-7348267/rs-pol...,2024-05-19,karumkit polri kramat jati brigjen hariyanto l...,0,netral,0,detik
2,jelas polri kemenhub pesawat latih jatuh bsd,https://news.detik.com/berita/d-7348256/penjel...,2024-05-19,pesawat latih jatuh kawasan bsd serpong tanger...,0,netral,0,detik
3,pesawat latih jatuh bsd evakuasi pakai crane,https://news.detik.com/berita/d-7348254/pesawa...,2024-05-19,puing puing pesawat latih pk ifp jatuh kawasan...,0,netral,0,detik
4,duka lepas salim said sang tokoh pers nasional,https://news.detik.com/berita/d-7348216/mereka...,2024-05-19,dunia pers duka prof salim said tutup usia pro...,0,netral,0,detik
...,...,...,...,...,...,...,...,...
95,ramai cfd sudirman jakpus pagi warga keluh sus...,https://news.detik.com/berita/d-7347306/ramain...,2024-05-19,kawasan jalan sudirman jalan mh thamrin jakart...,0,netral,0,detik
96,ria sakit siswi smp bogor korban rundung,https://news.detik.com/berita/d-7347299/teriak...,2024-05-19,siswi smp bojonggede kabupaten bogor jawa bara...,0,netral,0,detik
97,ribu warga jangkit dbd dprd sorot aspek tangan...,https://news.detik.com/berita/d-7347296/ribuan...,2024-05-19,anggota komisi dprd jawa timur bidang sehat de...,0,netral,0,detik
98,pimpin komisi dpr kritik kemendikbud didik wajib,https://news.detik.com/berita/d-7347292/pimpin...,2024-05-19,wakil ketua komisi dpr ri hetifah sjaifudian s...,0,netral,0,detik


In [21]:
df1 = df[df['content'].str.strip().astype(bool)]
df1

,title,link,date,content,is_fake,media_bias,bias,platform
0,polisi tunggu tuju keluarga korban pesawat jat...,https://news.detik.com/berita/d-7348286/polisi...,2024-05-19,jenazah korban pesawat latih jatuh bsd serpong...,0,netral,0,detik
1,rs polri kondisi korban tewas pesawat jatuh bs...,https://news.detik.com/berita/d-7348267/rs-pol...,2024-05-19,karumkit polri kramat jati brigjen hariyanto l...,0,netral,0,detik
2,jelas polri kemenhub pesawat latih jatuh bsd,https://news.detik.com/berita/d-7348256/penjel...,2024-05-19,pesawat latih jatuh kawasan bsd serpong tanger...,0,netral,0,detik
3,pesawat latih jatuh bsd evakuasi pakai crane,https://news.detik.com/berita/d-7348254/pesawa...,2024-05-19,puing puing pesawat latih pk ifp jatuh kawasan...,0,netral,0,detik
4,duka lepas salim said sang tokoh pers nasional,https://news.detik.com/berita/d-7348216/mereka...,2024-05-19,dunia pers duka prof salim said tutup usia pro...,0,netral,0,detik
...,...,...,...,...,...,...,...,...
95,ramai cfd sudirman jakpus pagi warga keluh sus...,https://news.detik.com/berita/d-7347306/ramain...,2024-05-19,kawasan jalan sudirman jalan mh thamrin jakart...,0,netral,0,detik
96,ria sakit siswi smp bogor korban rundung,https://news.detik.com/berita/d-7347299/teriak...,2024-05-19,siswi smp bojonggede kabupaten bogor jawa bara...,0,netral,0,detik
97,ribu warga jangkit dbd dprd sorot aspek tangan...,https://news.detik.com/berita/d-7347296/ribuan...,2024-05-19,anggota komisi dprd jawa timur bidang sehat de...,0,netral,0,detik
98,pimpin komisi dpr kritik kemendikbud didik wajib,https://news.detik.com/berita/d-7347292/pimpin...,2024-05-19,wakil ketua komisi dpr ri hetifah sjaifudian s...,0,netral,0,detik


In [22]:
df1 = df.dropna()
df1

,title,link,date,content,is_fake,media_bias,bias,platform
0,polisi tunggu tuju keluarga korban pesawat jat...,https://news.detik.com/berita/d-7348286/polisi...,2024-05-19,jenazah korban pesawat latih jatuh bsd serpong...,0,netral,0,detik
1,rs polri kondisi korban tewas pesawat jatuh bs...,https://news.detik.com/berita/d-7348267/rs-pol...,2024-05-19,karumkit polri kramat jati brigjen hariyanto l...,0,netral,0,detik
2,jelas polri kemenhub pesawat latih jatuh bsd,https://news.detik.com/berita/d-7348256/penjel...,2024-05-19,pesawat latih jatuh kawasan bsd serpong tanger...,0,netral,0,detik
3,pesawat latih jatuh bsd evakuasi pakai crane,https://news.detik.com/berita/d-7348254/pesawa...,2024-05-19,puing puing pesawat latih pk ifp jatuh kawasan...,0,netral,0,detik
4,duka lepas salim said sang tokoh pers nasional,https://news.detik.com/berita/d-7348216/mereka...,2024-05-19,dunia pers duka prof salim said tutup usia pro...,0,netral,0,detik
...,...,...,...,...,...,...,...,...
95,ramai cfd sudirman jakpus pagi warga keluh sus...,https://news.detik.com/berita/d-7347306/ramain...,2024-05-19,kawasan jalan sudirman jalan mh thamrin jakart...,0,netral,0,detik
96,ria sakit siswi smp bogor korban rundung,https://news.detik.com/berita/d-7347299/teriak...,2024-05-19,siswi smp bojonggede kabupaten bogor jawa bara...,0,netral,0,detik
97,ribu warga jangkit dbd dprd sorot aspek tangan...,https://news.detik.com/berita/d-7347296/ribuan...,2024-05-19,anggota komisi dprd jawa timur bidang sehat de...,0,netral,0,detik
98,pimpin komisi dpr kritik kemendikbud didik wajib,https://news.detik.com/berita/d-7347292/pimpin...,2024-05-19,wakil ketua komisi dpr ri hetifah sjaifudian s...,0,netral,0,detik


In [23]:
df1['gabung'] = df1['platform'] + ' ' + df1['title'] + ' ' + df1['content']
df1

,title,link,date,content,is_fake,media_bias,bias,platform,gabung
0,polisi tunggu tuju keluarga korban pesawat jat...,https://news.detik.com/berita/d-7348286/polisi...,2024-05-19,jenazah korban pesawat latih jatuh bsd serpong...,0,netral,0,detik,detik polisi tunggu tuju keluarga korban pesaw...
1,rs polri kondisi korban tewas pesawat jatuh bs...,https://news.detik.com/berita/d-7348267/rs-pol...,2024-05-19,karumkit polri kramat jati brigjen hariyanto l...,0,netral,0,detik,detik rs polri kondisi korban tewas pesawat ja...
2,jelas polri kemenhub pesawat latih jatuh bsd,https://news.detik.com/berita/d-7348256/penjel...,2024-05-19,pesawat latih jatuh kawasan bsd serpong tanger...,0,netral,0,detik,detik jelas polri kemenhub pesawat latih jatuh...
3,pesawat latih jatuh bsd evakuasi pakai crane,https://news.detik.com/berita/d-7348254/pesawa...,2024-05-19,puing puing pesawat latih pk ifp jatuh kawasan...,0,netral,0,detik,detik pesawat latih jatuh bsd evakuasi pakai c...
4,duka lepas salim said sang tokoh pers nasional,https://news.detik.com/berita/d-7348216/mereka...,2024-05-19,dunia pers duka prof salim said tutup usia pro...,0,netral,0,detik,detik duka lepas salim said sang tokoh pers na...
...,...,...,...,...,...,...,...,...,...
95,ramai cfd sudirman jakpus pagi warga keluh sus...,https://news.detik.com/berita/d-7347306/ramain...,2024-05-19,kawasan jalan sudirman jalan mh thamrin jakart...,0,netral,0,detik,detik ramai cfd sudirman jakpus pagi warga kel...
96,ria sakit siswi smp bogor korban rundung,https://news.detik.com/berita/d-7347299/teriak...,2024-05-19,siswi smp bojonggede kabupaten bogor jawa bara...,0,netral,0,detik,detik ria sakit siswi smp bogor korban rundung...
97,ribu warga jangkit dbd dprd sorot aspek tangan...,https://news.detik.com/berita/d-7347296/ribuan...,2024-05-19,anggota komisi dprd jawa timur bidang sehat de...,0,netral,0,detik,detik ribu warga jangkit dbd dprd sorot aspek ...
98,pimpin komisi dpr kritik kemendikbud didik wajib,https://news.detik.com/berita/d-7347292/pimpin...,2024-05-19,wakil ketua komisi dpr ri hetifah sjaifudian s...,0,netral,0,detik,detik pimpin komisi dpr kritik kemendikbud did...


In [24]:
df1 = df1[["gabung",'bias']]
# df1.rename(columns={'gabung': 'title'}, inplace=True)
df1

,gabung,bias
0,detik polisi tunggu tuju keluarga korban pesaw...,0
1,detik rs polri kondisi korban tewas pesawat ja...,0
2,detik jelas polri kemenhub pesawat latih jatuh...,0
3,detik pesawat latih jatuh bsd evakuasi pakai c...,0
4,detik duka lepas salim said sang tokoh pers na...,0
...,...,...
95,detik ramai cfd sudirman jakpus pagi warga kel...,0
96,detik ria sakit siswi smp bogor korban rundung...,0
97,detik ribu warga jangkit dbd dprd sorot aspek ...,0
98,detik pimpin komisi dpr kritik kemendikbud did...,0


In [25]:
df["gabung"] = df['platform'] + ' ' + df['title']
df

,title,link,date,content,is_fake,media_bias,bias,platform,gabung
0,polisi tunggu tuju keluarga korban pesawat jat...,https://news.detik.com/berita/d-7348286/polisi...,2024-05-19,jenazah korban pesawat latih jatuh bsd serpong...,0,netral,0,detik,detik polisi tunggu tuju keluarga korban pesaw...
1,rs polri kondisi korban tewas pesawat jatuh bs...,https://news.detik.com/berita/d-7348267/rs-pol...,2024-05-19,karumkit polri kramat jati brigjen hariyanto l...,0,netral,0,detik,detik rs polri kondisi korban tewas pesawat ja...
2,jelas polri kemenhub pesawat latih jatuh bsd,https://news.detik.com/berita/d-7348256/penjel...,2024-05-19,pesawat latih jatuh kawasan bsd serpong tanger...,0,netral,0,detik,detik jelas polri kemenhub pesawat latih jatuh...
3,pesawat latih jatuh bsd evakuasi pakai crane,https://news.detik.com/berita/d-7348254/pesawa...,2024-05-19,puing puing pesawat latih pk ifp jatuh kawasan...,0,netral,0,detik,detik pesawat latih jatuh bsd evakuasi pakai c...
4,duka lepas salim said sang tokoh pers nasional,https://news.detik.com/berita/d-7348216/mereka...,2024-05-19,dunia pers duka prof salim said tutup usia pro...,0,netral,0,detik,detik duka lepas salim said sang tokoh pers na...
...,...,...,...,...,...,...,...,...,...
95,ramai cfd sudirman jakpus pagi warga keluh sus...,https://news.detik.com/berita/d-7347306/ramain...,2024-05-19,kawasan jalan sudirman jalan mh thamrin jakart...,0,netral,0,detik,detik ramai cfd sudirman jakpus pagi warga kel...
96,ria sakit siswi smp bogor korban rundung,https://news.detik.com/berita/d-7347299/teriak...,2024-05-19,siswi smp bojonggede kabupaten bogor jawa bara...,0,netral,0,detik,detik ria sakit siswi smp bogor korban rundung
97,ribu warga jangkit dbd dprd sorot aspek tangan...,https://news.detik.com/berita/d-7347296/ribuan...,2024-05-19,anggota komisi dprd jawa timur bidang sehat de...,0,netral,0,detik,detik ribu warga jangkit dbd dprd sorot aspek ...
98,pimpin komisi dpr kritik kemendikbud didik wajib,https://news.detik.com/berita/d-7347292/pimpin...,2024-05-19,wakil ketua komisi dpr ri hetifah sjaifudian s...,0,netral,0,detik,detik pimpin komisi dpr kritik kemendikbud did...


In [26]:
df = df[["gabung",'bias']]

In [27]:
df

,gabung,bias
0,detik polisi tunggu tuju keluarga korban pesaw...,0
1,detik rs polri kondisi korban tewas pesawat ja...,0
2,detik jelas polri kemenhub pesawat latih jatuh...,0
3,detik pesawat latih jatuh bsd evakuasi pakai c...,0
4,detik duka lepas salim said sang tokoh pers na...,0
...,...,...
95,detik ramai cfd sudirman jakpus pagi warga kel...,0
96,detik ria sakit siswi smp bogor korban rundung,0
97,detik ribu warga jangkit dbd dprd sorot aspek ...,0
98,detik pimpin komisi dpr kritik kemendikbud did...,0


In [28]:
df = pd.concat([df,df1], axis=0, ignore_index=True)
df

,gabung,bias
0,detik polisi tunggu tuju keluarga korban pesaw...,0
1,detik rs polri kondisi korban tewas pesawat ja...,0
2,detik jelas polri kemenhub pesawat latih jatuh...,0
3,detik pesawat latih jatuh bsd evakuasi pakai c...,0
4,detik duka lepas salim said sang tokoh pers na...,0
...,...,...
195,detik ramai cfd sudirman jakpus pagi warga kel...,0
196,detik ria sakit siswi smp bogor korban rundung...,0
197,detik ribu warga jangkit dbd dprd sorot aspek ...,0
198,detik pimpin komisi dpr kritik kemendikbud did...,0


In [30]:
df.to_json('dataset_bias_detik_clean.json', orient='records', lines=True)